<a href="https://colab.research.google.com/github/SAQLAIN-MUSHTAQUE/HassanZindani-ResumeScreening/blob/main/Resume_ScreeningV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV Analyzer

## module install

In [ ]:
!apt-get update #Update the library of Linux
!apt-get install -y poppler-utils #Store temporary data of another library (PDF2image)
# !apt-get install -y libreoffice

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,163 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packa

In [ ]:
! pip install langchain_community docx2txt pinecone-client tiktoken langchain langchain-openai pypdf rapidocr-onnxruntime pdf2image loguru openai tiktoken langchain asyncio aiohttp  -qU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import os #Basic module
import requests
import asyncio #Making a parallel programming
import aiohttp
from concurrent.futures import ThreadPoolExecutor
from pdf2image import convert_from_path #PDF2image conversion
from PIL import Image #PDF2image conversion
import base64 #image format into binary format
from langchain_community.document_loaders import Docx2txtLoader
import io
import tempfile
from google.colab import userdata
from openai import AsyncOpenAI
from typing import Dict, List, Tuple, Any
from loguru import logger
import tiktoken
from google.colab import files
import json
import re
import uuid
import concurrent.futures
import nest_asyncio
import pandas as pd
from pinecone import  Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain_openai import OpenAIEmbeddings
from itertools import islice
from typing import Iterable, Iterator, List, Any, TypeVar, Dict, Tuple, Union, Optional
from collections import defaultdict

## OpenAI Credential

In [ ]:
# OpenAI Client
client = AsyncOpenAI(api_key = userdata.get('OPENAI_API_KEY'))

## **Loaders**

### PDF Reader Using OpenAI Vision (multiple file)

#### PDF to Image

In [ ]:
def pdf_page_to_image(pdf_path):
    """Convert PDF pages to images."""
    try:
      pdf = pdf_path.name
    except:
      pdf = pdf_path

    pages = convert_from_path(pdf, 300)  # 300 DPI for high quality
    return pages

#### OpenAI Vision Function

In [ ]:
async def encode_image(image_path):
    """Encode image as a base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

async def count_tokens_async(text, model="gpt-4o-mini"):
    """Count the number of tokens in a text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

async def send_image_to_openai_async(image_path, api_key, model="gpt-4o-mini"):
    """Send an image to OpenAI API and receive the extracted text."""
    base64_image = await encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    prompt = """Extract the exact text from the provided image without adding any additional lines at the beginning.
    """

    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 1200
    }

    async with aiohttp.ClientSession() as session:
        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
            response_data = await response.json()
            output_text = response_data['choices'][0]['message']['content']

    # Calculate tokens used
    input_tokens = await count_tokens_async(prompt, model=model)
    output_tokens = await count_tokens_async(output_text, model=model)
    total_tokens = input_tokens + output_tokens

    # Pricing details
    cost_per_input_token = 0.150 / 1_000_000  # $ per input token
    cost_per_output_token = 0.600 / 1_000_000  # $ per output token
    cost_per_image = 0.01275  # $ per image (300x300 px)

    # Calculate total cost
    input_cost = input_tokens * cost_per_input_token
    output_cost = output_tokens * cost_per_output_token
    total_cost = cost_per_image + input_cost + output_cost

    return {
        "content": output_text,
        "total_tokens": total_tokens,
        "total_cost": total_cost
    }

#### Text to Langchain Document

In [ ]:
def load_texts_with_sources(text_source_pairs):
    """
    Converts a list of text-source pairs into LangChain document format.

    Args:
        text_source_pairs (list of tuples): Each tuple contains (text, source).

    Returns:
        list of Document: LangChain documents with content and metadata.
    """
    documents = [
        Document(page_content=text, metadata={"source": source})
        for text, source in text_source_pairs
    ]
    return documents

#### Process image to text

In [ ]:
async def process_image_text(images, api_key, vision_model="gpt-4o-mini", filename="document.pdf"):
    """Process images and extract text as DocumentPage objects with metadata."""
    total_tokens = 0
    total_cost = 0

    async def process_single_image(image, image_index):
        with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp_file:
            image.save(temp_file, format="PNG")
            temp_file_path = temp_file.name

        # Process the image and extract text
        result = await send_image_to_openai_async(temp_file_path, api_key, vision_model)
        os.remove(temp_file_path)  # Remove the temporary file after processing

        # return text, total token and the total cost
        return result["content"], float(result["total_tokens"]), float(result["total_cost"])

    # Create tasks for each image
    tasks = [process_single_image(image, i) for i, image in enumerate(images, 1)]
    results = await asyncio.gather(*tasks)

    # Collect results
    all_text = ""
    for doc, tokens, cost in results:
        all_text += doc + "\n"
        total_tokens += tokens
        total_cost += cost


    # Converting into text pair
    text_source_pairs = [(all_text, filename)]

    # Converting into Langchain document
    documents = load_texts_with_sources(text_source_pairs)

    return documents, total_tokens, total_cost


### MS word loader

In [ ]:
def load_docx(file_path):
    loader = Docx2txtLoader(file_path)
    loader =  loader.load()
    return loader


## Text Splitter

In [ ]:
def text_seperator(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n\n", "\n\n\n", "\n\n", "\n• ", "\n○ ", "\n- ", "\t", "\n"],
        chunk_size=100,
        chunk_overlap = 15,
    )

    splitted_docs = text_splitter.split_documents(docs)


    return splitted_docs

## Process single file

In [ ]:
from sys import meta_path
# Processing Function single file
async def process_single_file(indx: int, filename: str, api_key: str, vision_model: str):
    """Asynchronously process a single file by converting to images and extracting text."""
    src_path = os.path.join("/content", filename)
    dest_path = os.path.join("/content/CV_folder", filename)
    os.rename(src_path, dest_path)

    # Get the file path
    file_path = dest_path
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    total_tokens = 0.0
    total_cost = 0.0

    if ext == ".docx":
        langchain_documents = load_docx(file_path)  # Assuming load_docx is implemented similarly with metadata
    elif ext == ".pdf":
        # Convert PDF pages to images
        images = pdf_page_to_image(file_path)
        print(f"Conversion for file {filename} has been done!")

        # Extract text from images asynchronously with Langchain-compatible output
        langchain_documents, total_tokens, total_cost = await process_image_text(images, api_key, vision_model=vision_model, filename=dest_path)
        print(f"Text extraction for file {filename} is complete!")

    # Raw Text
    all_text= ""
    for doc in langchain_documents:
      all_text += doc.page_content + "\n"

    # Splitting document
    splitted_docs = text_seperator(langchain_documents)

    # Key (for each file) to store extracted data
    key = dest_path

    result = {
        "splitted_docs": splitted_docs,
        "total_tokens": total_tokens,
        "total_cost": total_cost
    }

    return key, result, all_text

## Main orchestration Process

In [ ]:
# Main orcheshtration process (engine first part)
async def main_process(api_key: str, vision_model: str = "gpt-4o-mini"):
    """Main function to orchestrate the entire process asynchronously."""

    # Upload the file
    uploaded = files.upload()

    # Create the CV_folder if it doesn't exist
    os.makedirs('/content/CV_folder', exist_ok=True)

    # Initiate variables
    loaded_docs = {}
    all_raw_text = {}
    total_tokens = 0.0
    total_cost = 0.0

    # List to hold tasks for processing each file concurrently
    tasks = []

    # Move the uploaded files to /content/CV_folder and process them concurrently
    for indx, filename in enumerate(uploaded.keys()):
        task = process_single_file(indx, filename, api_key, vision_model)
        tasks.append(task)

    # Run all the tasks concurrently and gather the results
    results = await asyncio.gather(*tasks)
    # print(results)

    # Process the results
    for key, result, raw_text in results:
      loaded_docs[key] = result['splitted_docs']
      all_raw_text[key] = raw_text
      total_tokens += result['total_tokens']
      total_cost += result['total_cost']



    return loaded_docs, all_raw_text, total_tokens, total_cost

In [ ]:
loaded_docs,all_raw_text, total_tokens, total_cost = await main_process(userdata.get('OPENAI_API_KEY'))

Saving Affan Sami resume.pdf to Affan Sami resume.pdf
Saving H.M. Saqlain Mushtaque Resume 2023.pdf to H.M. Saqlain Mushtaque Resume 2023.pdf
Saving Murtaza_Resume.pdf to Murtaza_Resume.pdf
Saving Sharjeel CV.pdf to Sharjeel CV.pdf
Saving SM Ghuffran 2022.pdf to SM Ghuffran 2022.pdf
Conversion for file Affan Sami resume.pdf has been done!
Conversion for file H.M. Saqlain Mushtaque Resume 2023.pdf has been done!
Conversion for file Murtaza_Resume.pdf has been done!
Conversion for file Sharjeel CV.pdf has been done!
Conversion for file SM Ghuffran 2022.pdf has been done!
Text extraction for file Murtaza_Resume.pdf is complete!
Text extraction for file Sharjeel CV.pdf is complete!
Text extraction for file SM Ghuffran 2022.pdf is complete!
Text extraction for file Affan Sami resume.pdf is complete!
Text extraction for file H.M. Saqlain Mushtaque Resume 2023.pdf is complete!


## **Pinecone Upload**



### Pinecone Credentials

In [ ]:
PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# OpenAI Embedding model
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-large')

# Defining Index
pc = Pinecone(PINECONE_API_KEY)


### Define Index


In [ ]:
index = pc.Index(host=userdata.get('PINECONE_INDEX_URL'))

### Pinecone Upload Function

In [ ]:
T = TypeVar("T")

def batch_iterate(size: int, iterable: Iterable[T]) -> Iterator[List[T]]:
    """Utility batching function."""
    it = iter(iterable)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            return
        yield chunk
def upload_data(loaded_docs, index, embedding_model, namespace):
    embedding_chunk_size = 150
    batch_size = 70 # size of batch (length of chunks in one go.)

    ids_value = 0
    metadata_list = []
    texts = []
    ids = []

    for key, value in loaded_docs.items():
      texts.extend([page.page_content for page in value])
      metadata_list.extend([page.metadata for page in value])
      ids.extend([str(ids_value + i) for i in range(len(texts))])
      ids_value += len(texts)

    for metadata, text in zip(metadata_list, texts):
      metadata["text"] = text

    for i in range(0, len(texts), embedding_chunk_size):
      chunk_texts = texts[i : i + embedding_chunk_size]
      chunk_ids = ids[i : i + embedding_chunk_size]
      chunk_metadatas = metadata_list[i : i + embedding_chunk_size]
      embeddings = embedding_model.embed_documents(chunk_texts)

      # uploading asynchronously
      async_res = [
          index.upsert(
              vectors=batch,
              async_req=True,
              namespace=namespace
          )
          for batch in batch_iterate(
              batch_size, zip(chunk_ids, embeddings, chunk_metadatas)
          )
      ]
      [res.get() for res in async_res]

### Making Unique Namespace

In [ ]:
def generate_unique_namespace(base_name="CVs_data"):
    # Generate a unique index name by appending a UUID
    unique_id = uuid.uuid4().hex[:8]  # Get a short unique identifier
    return f"{base_name}_{unique_id}"

In [ ]:
# Define namespace
namespace = generate_unique_namespace()     #  In FastAPI it will be MongoDB Batch (Model) ID

In [ ]:
namespace = 'CVs_data_db716073'

### Uploading to Pinecone

In [ ]:
upload_data(loaded_docs, index, embedding_model, namespace)

NameError: name 'loaded_docs' is not defined

### Checking length of any namespace

In [ ]:
stats = index.describe_index_stats()

# Check the namespaces and their vector counts
namespace_stats = stats['namespaces']
count = namespace_stats.get(namespace,{}).get('vector_count', 0)
print(f"Number of entries in the namespace '{namespace}': {count}")

Number of entries in the namespace 'CVs_data_db716073': 148


## Query:

### Job Post

In [ ]:
query = """
As a DevOps Engineers at Dubizzle ensures, you will ensure our services are healthy, monitored, automated, and designed to scale. You'll use your background as an operations generalist to work closely with our development teams from the early stages of design all the way through identifying and resolving production issues. You will support a wide range of products focusing on automation, availability and performance, and above all reliability.


Requirements

Serve as a primary point responsible for the overall health, performance, efficiency and capacity of our production environment
Be part of the implementation and design of the systems used to operate Dubizzle, with a focus on automation and maintainability at large scale
Experience in building large CI/CD processes (experience with Github actions & Jenkins is preferred)
Collaboration with the development team on operations-related issues, providing support and acting as stakeholder
Provide fast and reliable development experience for product teams by maintaining production-like environments used for development and testing
Develop tools to effectively monitor custom applications in a large-scale environment
Troubleshoot issues across the entire stack - hardware, software, application and network
Migrate applications off of legacy environments with minimal downtime
Take part in a shared 24x7 on-call rotation
Document system design and procedures
Experience & Qualification

Bachelor's Degree in Computer Science, Engineering or equivalent field and significant experience in DevOps
AWS Solutions Architect certification
4+ years experience in supporting hosted services in a high-volume customer-facing environment
Experience with DevOps tools, processes, culture, distributed systems, supporting mission-critical systems
Experience working on Continuous Integration / Delivery pipelines using tools such as Jenkins and Github actions
Demonstrated ability to write programs / utilities using a high-level programming language like: Go, Ruby, Python as well as Bash
3+ years proven experience in running and managing infrastructure at scale using container orchestration tool kubernetes
Experience in performing migration from traditional technology stack(s) to AWS EKS or similar


Benefits



fast paced, high performing team
ulticultural environment with over 40 different nationalities
omprehensive Health insurance (Spouse & Children)
ewards & Recognitions
earning & Development opportunities
ym Allowance
aptop Ownership
aid Leaves

Benefits found in job post

Medical insurance
"""

### Quary System Prompt

In [ ]:
query_prompt = """
You are an excellent Job Post Analyzer and vector database query writer. You will provided a job post and your job is to analyze it and extract the following information that can be used to search the vector database.

**IMPORTANT:** Write the information in your response like that can be used to search the vector database.

Information_fields:
job_title: The title of the job. It must be present in the job posting. It should be string "". This information is compulsory and not empty, and if there is no job title, then this is not a valid job post, and in that case, simply return "Invalid Job Post" without searching for the further field's information.
job_description: The description of the job. It must be present in the job posting. It should be string "". If there is no job description, then return None.
required_experience: The experience or the level of the candidate required for the job. This information may describe the level of experience or the duration of experience that candidate must have. If there is more than one required experience, then write the job role with the required number of durations of experience. This information should be the string "". This information must be present in the job posting. If this information is not present, then write your remarks about the experience according to the Job.
job_reponsibilities_required: The list of responsibilities or the roles for the given job that candidate must have. This information should be written in comma separated value inside the string "". This information must be present in the job post. If this information is not present, then write your remarks about the responsibilities according to the Job post.
job_reponsibilities_optional: The list of responsibilities or the roles for the given job that candidate can have. This information should be written in comma separated value inside the string "". This information may be present in the job posting. If there is no optional responsibilities, then return None.
required_education: The list of education required for the job. This information should be written in comma separated value inside the string "". This information must be present in the job posting. If this information is not present, then write your remarks about the education according to the Job post.
required_degree: The degree or list of qualifications required for the job. This information should be written in comma separated value inside the string "". If there is no degree required, then return None.
required_skills: The list of skills required for the job. This information should be written in comma separated value inside the string "". This information must be present in the job posting. If this information is not present, then write your remarks about the skill for the given Job post.
required_location: The requirement of location for the job. This information should be written in comma separated value inside the string "". This information is not compulsory. If there is no location required, then return None.
extra_info: All the other information that are not listed in the above fields. This information should be in detail without leaving any single information that are not addressed above. This information should be written in comma separated value inside the string "". This information may be present in the job posting. If there is no extra information, then return None.

Your output should be in the JSON format:
{{
  "job_title": "",
  "job_description": "" || None,
  "required_experience": "",
  "job_reponsibilities_required": "",
  "job_reponsibilities_optional": "" || None,
  "required_education": "",
  "required_degree": "" || None,
  "required_skills": "" ,
  "required_location": "" || None,
  "extra_info": "" || None
}}

"""

### Pass to the LLM (OpenAI)

In [ ]:
async def extract_query_info(client, query_system_prompt, query, llm_model = "gpt-4o-2024-08-06"):


   # User message
    user_message = f"""Extract the information from the job post text: {query}"""

    # Generate the completion
    completion = await client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": query_system_prompt},
            {"role": "user", "content": user_message},
        ],
        response_format={ "type": "json_object" },
    )

    # Get the response message
    json_message = completion.choices[0].message.content
    message = json.loads(json_message)

    return message

In [ ]:
extracted_query_info= await  extract_query_info(client, query_system_prompt = query_prompt, query= query, llm_model = "gpt-4o-2024-08-06")

In [ ]:
extracted_query_info

{'job_title': 'DevOps Engineer',
 'job_description': "As a DevOps Engineers at Dubizzle ensures, you will ensure our services are healthy, monitored, automated, and designed to scale. You'll use your background as an operations generalist to work closely with our development teams from the early stages of design all the way through identifying and resolving production issues. You will support a wide range of products focusing on automation, availability and performance, and above all reliability.",
 'required_experience': '4+ years experience in supporting hosted services in a high-volume customer-facing environment, 3+ years proven experience in running and managing infrastructure at scale using container orchestration tool kubernetes',
 'job_reponsibilities_required': 'Serve as a primary point responsible for the overall health, performance, efficiency and capacity of our production environment, Be part of the implementation and design of the systems used to operate Dubizzle, Collabo

### Filling the query info in the variables

In [ ]:
job_data = {
    'job_title': extracted_query_info.get('job_title'),
    'job_description': extracted_query_info.get('job_description', ''),
    'required_experience': extracted_query_info.get('required_experience'),
    'job_responsibilities_required': extracted_query_info.get('job_reponsibilities_required'),
    'job_responsibilities_optional': extracted_query_info.get('job_reponsibilities_optional', ''),
    'required_education': extracted_query_info.get('required_education'),
    'required_degree': extracted_query_info.get('required_degree', ''),
    'required_skills': extracted_query_info.get('required_skills'),
    'required_location': extracted_query_info.get('required_location', ''),
    'extra_info': extracted_query_info.get('extra_info', '')
}


### Utils: Supporting Function


In [ ]:
# Supporting Functions
def transform_and_sort_response(response):
    # Transform data to include text and sum of scores
    transformed_data = {
        file_path: {
            'text': details['text'],
            'score': max(details['score'])
        }
        for file_path, details in response.items()
    }

    # Sort the transformed data by score_sum in descending order
    sorted_data = dict(sorted(transformed_data.items(), key=lambda item: item[1]['score'], reverse=True))
    return sorted_data

def tok_k_function(index,namespace):
    # Calculating top-k using index
    stats = index.describe_index_stats()
    # check the namespaces and their vector counts
    namespace_stats = stats['namespaces']
    namespace= namespace
    experience_top_k= namespace_stats.get(namespace,{}).get('vector_count', 0)

    return experience_top_k

# Helper function to handle querying and extracting matching
async def query_matching(query: str, namespace: str, top_k: int):
    """
    Queries the Pinecone service with the provided query string, namespace, and top_k value.

    Args:
        query (str): The query string to search for.
        namespace (str): The namespace in Pinecone to query against.
        top_k (int): The number of top results to return.

    Returns:
        list: A list of fund dictionaries returned from Pinecone.
    """
    return await query_pinecone(query=query, namespace=namespace, top_k=top_k)


# Define top_k
top_k = tok_k_function(index,namespace)

async def filtering_other_field(job_query, filter_list, max_score, top_k:int=top_k):
    tasks = [
        query_pinecone(query=job_query, namespace=namespace, top_k=top_k, filter={"source": {"$eq": name}})
        for name in filter_list
    ]

    results = []
    for completed_task in asyncio.as_completed(tasks):
        result = await completed_task
        results.append(result)  # Collect results as they complete

    flattened_results = [item for sublist in results for item in sublist]  # Flatten the list of lists

    final_results = {}

    # Filtering score
    for item in flattened_results:
        if item['score'] > max_score:
            file_name = item['metadata']['source']
            if file_name not in final_results:
                final_results[file_name] = {}
                final_results[file_name]['score'] = [item['score']]
                final_results[file_name]['reason'] = f"The field matched with the field of the candidate"
                final_results[file_name]['text'] = item['metadata']['text']
            else:
                final_results[file_name]['score'].append(item['score'])
                final_results[file_name]['text'] += " " + item['metadata']['text']

    # Transform recent dictionary
    transformed_dict = transform_and_sort_response(final_results)

    return transformed_dict

# Merging Dictionary Function
def merge_dictionaries(scoring_dict, *other_dicts):
    scoring_dict_copy = scoring_dict.copy()  # Create a copy to avoid modifying the original
    for data_dict in other_dicts:
        for filename, data in data_dict.items():
            if filename in scoring_dict_copy:
                # Append the new text to the existing text
                scoring_dict_copy[filename]['text'] += ' ' + data['text']
                # Add the new score to the existing score
                scoring_dict_copy[filename]['score'] += data['score']
            else:
                # Create a new entry if the filename does not exist in scoring_dict_copy
                scoring_dict_copy[filename]['text'] = data['text']
                scoring_dict_copy[filename]['score'] = data['score']

    for filename, value in scoring_dict_copy.items():
        print(f"Before increment: {value['score']}")
        scoring_dict_copy[filename]['score'] = (value['score']/0.5)
        print(f"After increment: {value['score']}")

    return scoring_dict_copy


# Function to Add CV Text into merged_dict
def adding_cv_text(merged_scoring_dict, all_raw_text):
  for filename, data in all_raw_text.items():
    if filename in merged_scoring_dict:
      merged_scoring_dict[filename]['CV_text'] = data
  return merged_scoring_dict

### Query Function

In [ ]:
# Define top_k
top_k = tok_k_function(index,namespace)
async def query_pinecone(query: str, namespace: str = namespace, index = index, top_k: int= top_k , filter: dict = None) -> list:
    """
    Query the Pinecone index with the given parameters.

    Args:
        query: The query string to be embedded and searched.
        namespace: The namespace to query within.
        top_k: The number of top results to return.
        filter: Optional dictionary for metadata filtering.

    Returns:
        list:The query responses from Pinecone.
    """
    # Embed the query string
    loop = asyncio.get_running_loop()
    with ThreadPoolExecutor() as pool:
        # Embedding in thread pool
        embedded = await loop.run_in_executor(pool, embedding_model.embed_query, query)

        # Pinecone query in thread pool
        query_params = {
            "namespace": namespace,
            "vector": embedded,
            "top_k": top_k,
            "include_metadata": True,
        }
        if filter:
            query_params["filter"] = filter

        try:
            responses = await loop.run_in_executor(pool, lambda: index.query(**query_params))
        except Exception as e:
            print(f"Error during Pinecone query: {e}")
            return []

    # Extract essential data
    results = []
    for match in responses["matches"]:
        # print (f"{query} \n {match['id']}")
        results.append({
            "id": match["id"],
            "score": match["score"],
            "metadata": match["metadata"],
            "values":match["values"]
        })
    return results  # Return a list of dictionaries instead of Pinecone matches




### Filtering

In [ ]:
async def filtering_skill_edu_exp(job_data, top_k = 1):

  # Create asynchronous tasks for fund queries
  if 'required_experience' in job_data and job_data['required_experience'] != "":
    query = job_data['required_experience']
    task1 = [query_pinecone(query = query)]

      # 'skill_matching': query_matching(diversity_focused, "funds-diversity-focus", top_k_diversity_focus),
  if 'required_education' in job_data and job_data['required_education'] != "":
    query = job_data['required_education']

    task2 = [query_pinecone(query = query)]

  if 'required_skills' in job_data and job_data['required_skills'] != "":
    query = job_data['required_skills']
    task3 = [query_pinecone(query = query)]

  if 'job_responsibilities_required' in job_data and job_data['job_responsibilities_required'] != "":
    query = job_data['job_responsibilities_required']
    task4 = [query_pinecone(query = query)]

  # Run task1 and task2 concurrently
  raw_results1, raw_results2, raw_results3, raw_results4 = await asyncio.gather(
      asyncio.gather(*task1),  # Gather all results from task1
      asyncio.gather(*task2),  # Gather all results from task2
      asyncio.gather(*task3),   # Gather all results from task3
      asyncio.gather(*task4)   # Gather all results from task4
  )
  results1 = [item for sublist in raw_results1 for item in sublist]  # Flatten the list of lists
  results2 = [item for sublist in raw_results2 for item in sublist]  # Flatten the list of lists
  results3 = [item for sublist in raw_results3 for item in sublist]  # Flatten the list of lists
  results4 = [item for sublist in raw_results4 for item in sublist]  # Flatten the list of lists

  final_results = {}

  for r1 in results1:
      file_name = r1['metadata']['source']
      if r1['score'] >=0.5:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['score'] = [r1['score']]
              final_results[file_name]['reason'] = f"The job experience field matched with the experience of the candidate"
              final_results[file_name]['text'] = r1['metadata']['text']
          else:
              final_results[file_name]['score'].append(r1['score'])
              final_results[file_name]['text'] += " " + r1['metadata']['text']

  for r2 in results2:
      file_name = r1['metadata']['source']
      if r2['score'] >=0.5:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['score'] = [r2['score']]
              final_results[file_name]['reason'] = f"The education matched with the education of the candidate"
              final_results[file_name]['text'] = r2['metadata']['text']
          else:
              final_results[file_name]['score'].append(r2['score'])
              final_results[file_name]['text'] += " " + r2['metadata']['text']

  for r3 in results3:
      file_name = r3['metadata']['source']
      if r3['score'] >=0.5:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['score'] = [r3['score']]
              final_results[file_name]['reason'] = f"The skills matched with the skills of the candidate"
              final_results[file_name]['text'] = r3['metadata']['text']
          else:
              final_results[file_name]['score'].append(r3['score'])
              final_results[file_name]['text'] += " " + r3['metadata']['text']

  for r4 in results4:
    file_name = r4['metadata']['source']
    if r4['score'] >=0.5:
        if file_name not in final_results:
            final_results[file_name] = {}
            final_results[file_name]['score'] =[r4['score']]
            final_results[file_name]['reason'] = f"The responsibilities matched with the responsibilities of the candidate"
            final_results[file_name]['text'] = " " + r4['metadata']['text']
        else:
            final_results[file_name]['score'].append(r4['score'])
            final_results[file_name]['text'] += r4['metadata']['text']

  return final_results


In [ ]:
pf_result= await filtering_skill_edu_exp(job_data,index)

In [ ]:
pf_result

{'/content/CV_folder/SM Ghuffran 2022.pdf': {'score': [0.538154781],
  'reason': 'The skills matched with the skills of the candidate',
  'text': '• Development Tools: Git, Docker, Jenkins, JIRA, Bitbucket, Visual Studio, Eclips'},
 '/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'score': [0.528632641,
   0.526331604],
  'reason': 'The skills matched with the skills of the candidate',
  'text': 'CI/CD  REST API  Docker  MS Office  \nAWS Services  Problem Solver SKILLS:\nPython SQL and NoSQL Linux GitHub\nData Structure and Algorithm Data Analysis\nCI/CD REST API Docker MS Office\nAWS Services Problem Solver'}}

In [ ]:
scoring_dict = transform_and_sort_response(pf_result)
scoring_dict

{'/content/CV_folder/SM Ghuffran 2022.pdf': {'text': '• Development Tools: Git, Docker, Jenkins, JIRA, Bitbucket, Visual Studio, Eclips',
  'score': 0.538154781},
 '/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'text': 'CI/CD  REST API  Docker  MS Office  \nAWS Services  Problem Solver SKILLS:\nPython SQL and NoSQL Linux GitHub\nData Structure and Algorithm Data Analysis\nCI/CD REST API Docker MS Office\nAWS Services Problem Solver',
  'score': 0.528632641}}

### Filtering against remaining fields

In [ ]:
# Filter list
filter_list = list(scoring_dict.keys())

In [ ]:
# Comparing job_title
if "job_title" in job_data and (job_data["job_title"] != "" and job_data["job_title"] != None):
  query = job_data["job_title"]
  title_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (title_matched)
else:
  title_matched = {}

{'/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'text': "CI/CD  REST API  Docker  MS Office  \nAWS Services  Problem Solver SKILLS:\nPython SQL and NoSQL Linux GitHub\nData Structure and Algorithm Data Analysis\nCI/CD REST API Docker MS Office\nAWS Services Problem Solver H.M. Saqlain Mushtaque  \nDevOps Trainee  \nmushtaqesaqlain@gmail.com  \n+92 301 2335058  \nKarachi, Pakistan  \nlinkedin.com/in/saqlain-mushtaque-065058b9/  \ngithub.com/SAQLAIN-MUSHTAQUE \n\nWORK EXPERIENCE:  \n- Cloud DevOps Trainee  \nSkipQ, Remote  \nJan, 2023 – May, 2023  \nAchievement/Tasks  \n- Used Infrastructure-as-Code (IaC) constructs to build and operate a full stack production grade web application.  \n- Wrote RESTful Python application on AWS and developed automated CI/CD pipeline and also automated metrics to roll-back a deployment in case of service degradation.  \n- Designed multiple applications and implemented some AWS services using AWS Lambda, ALB, SNS, SQS, DynamoDB, Code-Pipeline

In [ ]:
# Comparing job_description
if "job_description" in job_data and (job_data["job_description"] != "" and job_data["job_description"] != None):
  query = job_data["job_description"]
  description_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (description_matched)
else:
  description_matched = {}

{'/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'text': "H.M. Saqlain Mushtaque  \nDevOps Trainee  \nmushtaqesaqlain@gmail.com  \n+92 301 2335058  \nKarachi, Pakistan  \nlinkedin.com/in/saqlain-mushtaque-065058b9/  \ngithub.com/SAQLAIN-MUSHTAQUE \n\nWORK EXPERIENCE:  \n- Cloud DevOps Trainee  \nSkipQ, Remote  \nJan, 2023 – May, 2023  \nAchievement/Tasks  \n- Used Infrastructure-as-Code (IaC) constructs to build and operate a full stack production grade web application.  \n- Wrote RESTful Python application on AWS and developed automated CI/CD pipeline and also automated metrics to roll-back a deployment in case of service degradation.  \n- Designed multiple applications and implemented some AWS services using AWS Lambda, ALB, SNS, SQS, DynamoDB, Code-Pipeline and etc. CI/CD  REST API  Docker  MS Office  \nAWS Services  Problem Solver SKILLS:\nPython SQL and NoSQL Linux GitHub\nData Structure and Algorithm Data Analysis\nCI/CD REST API Docker MS Office\nAWS Services Probl

In [ ]:
# Comparing job_reponsibilities_optional
if "job_responsibilities_optional" in job_data and (job_data["job_responsibilities_optional"] != "" and job_data["job_responsibilities_optional"] != None):
  query = job_data["job_responsibilities_optional"]
  optional_responsibilities_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (optional_responsibilities_matched)
else:
  optional_responsibilities_matched = {}

In [ ]:
# Comparing required_degree
if "required_degree" in job_data and (job_data["required_degree"] != "" and job_data["required_degree"] != None):
  query = job_data["required_degree"]
  degree_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (degree_matched)
else:
  degree_matched = {}

{'/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'text': "SKILLS:\nPython SQL and NoSQL Linux GitHub\nData Structure and Algorithm Data Analysis\nCI/CD REST API Docker MS Office\nAWS Services Problem Solver CI/CD  REST API  Docker  MS Office  \nAWS Services  Problem Solver \n\nWORK EXPERIENCE:  \n- Cloud DevOps Trainee  \nSkipQ, Remote  \nJan, 2023 – May, 2023  \nAchievement/Tasks  \n- Used Infrastructure-as-Code (IaC) constructs to build and operate a full stack production grade web application.  \n- Wrote RESTful Python application on AWS and developed automated CI/CD pipeline and also automated metrics to roll-back a deployment in case of service degradation.  \n- Designed multiple applications and implemented some AWS services using AWS Lambda, ALB, SNS, SQS, DynamoDB, Code-Pipeline and etc. \n\nI have graduated with a Space Science degree and a passion for cloud computing, machine learning, artificial intelligence, and data science, along with experience in Cloud Dev

In [ ]:
# Comparing location
if "required_location" in job_data and (job_data["required_location"] != "" and job_data["required_location"] != None):
  query = job_data["required_location"]
  location_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (location_matched)
else:
  location_matched = {}

In [ ]:
# Comparing Extra info
if "extra_info" in job_data and (job_data["extra_info"] != "" and job_data["extra_info"] != None):
  query = job_data["extra_info"]
  extra_info_matched = await filtering_other_field(query, filter_list, max_score=0.3)
  print (extra_info_matched)
else:
  extra_info_matched = {}

{'/content/CV_folder/SM Ghuffran 2022.pdf': {'text': 'WORK EXPERIENCE\nBRB Group\nML ENGINEER\nJanuary 2022 - February 2024', 'score': 0.314922392}, '/content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf': {'text': '\n\n- Assistant Consultant and Back Office Worker  \nPowerprep International, Karachi  \nMay, 2023 – Present  \nAchievement/Tasks  \n- Provide information, end to end service while organizing documents for visa and creating the itinerary.  \n- Identify the websites and online resources that are relevant for data collection using python programming and exploring data using MS-Excel.', 'score': 0.30523175}}


### Merging with scoring dictionary

In [ ]:
extra_info_matched
location_matched
degree_matched
optional_responsibilities_matched
description_matched
title_matched

# Merge title_matched and description_matched into scoring_dict
merged_scoring_dict = merge_dictionaries(scoring_dict,
                                         title_matched,
                                         description_matched,
                                         optional_responsibilities_matched,
                                         degree_matched,
                                         location_matched,
                                         extra_info_matched
                                         )


Before increment: 2.084661514
After increment: 4.169323028
Before increment: 2.2067044350000002
After increment: 4.4134088700000005


In [ ]:
for key, value in merged_scoring_dict.items():
  print(f"key: {key}, score: {value['score']}")

key: /content/CV_folder/SM Ghuffran 2022.pdf, score: 4.169323028
key: /content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf, score: 4.4134088700000005


In [ ]:
# Adding Cv Text into merged dict
merged_scoring_dict = adding_cv_text(merged_scoring_dict, all_raw_text)
merged_scoring_dict

NameError: name 'all_raw_text' is not defined

## **Evaluation by LLM**

### Evaluation system Prompt

In [ ]:
evaluate_prompt = """
You are an excellent CV evaluator. You have the job_post and are provided with CV text.
Your job is to evaluate the CV based on the post and return a matching score (0-5) along with the some information from the CV.

**Important:** The score range is between 0 to 5, with specific guidelines below.

job_post = {job_post}

Score Assessment Guidelines:
1. **Score 5: Overqualified**
    - **Experience**: Exceeds required years of experience by a significant margin.
    - **Skills**: Possesses all required skills and more; may include expertise in advanced or related skills beyond job requirements.
    - **Certifications and Courses**: Holds specialized or advanced certifications in the field, often beyond typical qualifications.
    - **Education**: Degree aligns closely with the field and is advanced or specialized for the role.
    - **Other Aspects**: Demonstrates notable industry achievements or contributions that indicate leadership, innovation, or high recognition.

2. **Score 4: Highly Qualified**
    - **Experience**: Meets or slightly exceeds experience requirements.
    - **Skills**: Covers all required skills well, possibly with some additional relevant skills.
    - **Certifications and Courses**: Holds relevant certifications or recent training specific to key skills.
    - **Education**: Degree aligns well with the role and may be at a higher level or specialized.
    - **Other Aspects**: Displays proactive industry engagement, relevant publications, or a record of strong performance indicators.

3. **Score 3: Qualified**
    - **Experience**: Meets the minimum experience requirement but doesn’t exceed significantly.
    - **Skills**: Has the primary skills required for the job, though without strong breadth beyond essentials.
    - **Certifications and Courses**: May have some relevant courses or certifications, though not extensive or advanced.
    - **Education**: Degree aligns reasonably well with job requirements.
    - **Other Aspects**: Demonstrates a consistent record of performance, stability, and role relevance.

4. **Score 2: Partially Qualified**
    - **Experience**: Slightly under the required years but relevant, showing potential for growth.
    - **Skills**: Has some of the core skills but lacks proficiency in a few key areas.
    - **Certifications and Courses**: Lacks certifications specific to the role; few relevant courses.
    - **Education**: Degree is related but general, not specialized.
    - **Other Aspects**: Shows interest or passion in the industry but lacks strong supporting achievements.

5. **Score 1: Minimally Qualified**
    - **Experience**: Considerably under the experience requirement.
    - **Skills**: Only partially matches the required skills; major gaps are apparent.
    - **Certifications and Courses**: No relevant certifications or courses.
    - **Education**: Degree is unrelated or significantly different from job requirements.
    - **Other Aspects**: Lacks industry engagement or notable achievements; minimal evidence of alignment with the role.

6. **Score 0: Not Qualified**
    - **Experience**: No relevant experience in the field.
    - **Skills**: Lacks any of the core skills necessary for the job.
    - **Certifications and Courses**: No certifications or courses relevant to the field.
    - **Education**: Educational background unrelated to the field.
    - **Other Aspects**: No signs of relevant industry interest, engagement, or potential for growth in the role.

**IMPORTANT:** Provide the reason for your rating. The response of this field should be in string.

Some Information from CV:
1. full_name: Full name of the candidate.
2. contact_number: List the contact number of the candidate, if it is available otherwise write "not_mentioned".
3. emial: Email of the candidate, if it is available otherwise write "not_mentioned".
4. address: Address of the candidate, if it is available otherwise write "not_mentioned".
5. highest_degree: Highest degree of the candidate. If there is no such degree then return None.
6. bachelors_degree: Bachelors degree or undergraduate degree or associate degree with the number of years, if available. If there is no such degree then return None.
7. masters_degree: Masters degree or postgraduate degree with the number of years, if available. If there is no such degree then return None.
8. certifications: All the certifications of the candidate, if available. Return the cetificate name, organization (if available), like: [<certificate1-organization>, <certificate2-organization>, <certificate3>, ...]. If there is no such degree then return None.
9. total_years_of_experience: The list of field with the total years of experience of the candidate. If there is no experience then return None. The response of this information should comma separated in the string like: "<field1>:<duration of experience, <field2>:<duration of experience>, ...".
10. skills: A section highlighting both hard skills (technical abilities) and soft skills (interpersonal abilities) that candidate mentioned. If there is no skills then return None.
11. linkedin_links: The linkedin profile link of the candidate, if available. If there is no linkedin link then return None.
12. achievements_and_awards: A list of achievements and awards earned by the candidate. If there is no achievements and awards then return None.
13. projects: A list of the personal or professional projects that the candidate made. If there is no projects then return None.
14. publications: A list of the articles or papers that candidate have authored. If there is no publications then return None .
15. gitHub_link: The github link of the candidate, if available. If there is no github link then return None.
16. Website_link:The portfolio website link of the candidate, if available. If there is no website link then return None.
17. languages: A list of languages that candidate speak, along with your proficiency levels. The value this field should be the list dictionary, where every dictionary has the language as the key and the proficiency level as the value. If there is no languages then return  "not_mentioned".

**IMPORTANT:** All links should be written with the prefix 'https://'.

**Output format:**
Return your response in JSON format using the structure below:

{{  "score": int(),
    "reason": "",
    "full_name": "" || "not_mentioned",
    "contact_number": [] || "not_mentioned",
    "emial": "" || "not_mentioned",
    "address": "" || "not_mentioned",
    "highest_degree": "" || None,
    "bachelors_degree": "" || None,
    "masters_degree": "" || None,
    "certifications": [] || None,
    "total_years_of_experience": "" || None,
    "skills": [] || None,
    "linkedin_links": "" || None,
    "achievements_and_awards": [] || None,
    "projects": [] || None,
    "publications": [] || None,
    "gitHub_link": "" || None,
    "Website_link": "" || None,
    "languages": [{{}}] || "not_mentioned",
}}
"""



### Pass to the LLM (OpenAI)

In [ ]:
async def evaluate_cv(client, evaluate_prompt, job_post, resume_text, llm_model = "gpt-4o-2024-08-06"):


   # User message
    user_message = f"""Evaluate and extract the information from the resume text: {resume_text}"""

    # Generate the completion
    completion = await client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": evaluate_prompt.format(job_post = job_post)},
            {"role": "user", "content": user_message},
        ],
        response_format={ "type": "json_object" },
    )

    # Get the response message
    json_message = completion.choices[0].message.content
    message = json.loads(json_message)

    return message

In [ ]:
async def llm_evaluation(client, merged_scoring_dict, job_post, evaluate_prompt, llm_model="gpt-4o-2024-08-06"):
    final_dictionary = {}

    # Creating a list of tasks to run concurrently
    tasks = [
        evaluate_cv(client, evaluate_prompt, job_post, value['CV_text'])
        for file_name, value in merged_scoring_dict.items()
    ]

    # Await all tasks concurrently
    responses = await asyncio.gather(*tasks)

    # Populating the final dictionary with responses
    for (file_name, value), response in zip(merged_scoring_dict.items(), responses):
        response['score'] += value['score']  # Adding the previous score
        response['score'] = round((response['score']/10)*100, 2)
        final_dictionary[file_name] = response
        final_dictionary[file_name]['CV_text'] = value['CV_text']

    # Sorting the dictionary by score in descending order
    sorted_final_dictionary = dict(
        sorted(final_dictionary.items(), key=lambda item: item[1]['score'], reverse=True)
    )

    return sorted_final_dictionary

### **Passing with LLM To Evaluate**

In [ ]:
final_result = await llm_evaluation(client, merged_scoring_dict, query, evaluate_prompt)
final_result

{'/content/CV_folder/SM Ghuffran 2022.pdf': {'score': 67.89,
  'reason': "The candidate is highly qualified with relevant experience and skills in line with the job post's requirements. They have over two years of experience in fields related to machine learning engineering and web development with strong skills in programming and development tools. However, there is no mention of multicultural or international exposure, certifications, or advanced degrees that would suggest overqualification.",
  'full_name': 'SYED MUHAMMAD GHUFFRAN',
  'contact_number': ['+92347-2360709'],
  'emial': 'ghuffi016@gmail.com',
  'address': 'Gulistan e Johar Block 7 Karachi',
  'highest_degree': 'BE Computer System Engineering',
  'bachelors_degree': 'BE Computer System Engineering:4',
  'masters_degree': None,
  'certifications': None,
  'total_years_of_experience': 'ML ENGINEER:2, JUNIOR PYTHON DEVELOPER:0.5',
  'skills': ['Embedded Systems',
   'RTOS',
   'Microcontrollers (ARM, ESP32)',
   'IoT protoc